# Explainability

## Feature Importance

We can look to quantify how much influence a single feature has on a label prediction. Explainers work by evaluating a test data set of feature cases and the labels it predicts for them.

### Globale Feature Importance

Global feature importance quanitifies the importance of each feature in the test dataset as a whole. it provides a general comparison of the extent to which each feature influences the prediction.

### Local Feature Importance

Local Feature importance measures the influence of each feature value for a specific individual prediction.

In a binary classification model, each feature gets a local importance value for each possible class, showing the amount of support for each class that it can predict. 

#### Mismatch between Global and Local features

There could be multiple reasons why local importance for an individual prediction varies from global importance for the entire dataset. 

## Using Explainers

You can use AML SDK to create explainers for ML models, even if the model was not trained using an experiment.

### Creating an explainer
to interpret a locally created model, you must install the **azureml-interpret** package. You can use this package to create an explainer. 

There are multiple types of explainers:
- MimicExplainer
    - This is an explainer that creates a global surrogate model that approximates your trained model and can be used to generate explanations. This surrogate model must have the same architecture as your model (e.g. linear or tree based)
-  TabularExplainer
    - An explainer that acts as a wrapper around various SHAP explainer algorithms, automatically choosing the one that is most appropriate for your model architecture
- PFIExplainer
    - A permutation Feature Importance explainer that analyses feature importance by shuffling feature values and measuring the impact on prediction performance

imagine we have a hypothetical model called `loan_model`, lets create the above explainer types:

## Explaining Global Feature Importance

To retreive global importance values, you call **explain_global()** method of your explainer to get a global explanation, then you can use the **get_feature_importance_dict()** method to get a dictionary of feature importance values. 

## Explaining local feature importance

to generate local feature importance from a MimicExplainer or TabularExplainer, you must call **explain_local()** method.

You can use **get_ranked_local_names()** and **get_ranked_local_values()** methods to retreive dictionaries of the feature names & values, ranked by importance.

In [ ]:
# MimicExplainer
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import DecisionTreeExplainableModel

mim_explainer = MimicExplainer(
    model = loan_model,
    initialization_examples = X_test,
    explainable_model = DecisionTreeExplanableModel,
    features = ['loan_amount','income','age','marital_status'],
    classes = ['reject', 'approve']
)
# global feature importance
global_mim_explainer = mim_explainer.explain_global(X_test)
global_mim_feature_importance = global_mim_explanation.get_feature_importance_dict()
# local feature importance
local_mim_explanation = mim_explainer.explain_local(X_test[0:5])
local_mim_features = local_mim_explanation.get_ranked_local_names()
local_mim_importance = local_mim_explanation.get_ranked_local_values()



# TabularExplainer
from interpret.ext.blackbox import TabularExplainer 

tab_explainer = TabularExplainer(
    model = loan_model,
    initialization_examples = X_test,
    features = ['loan_amount','income','age','marital_status'],
    classes = ['reject', 'approve']
)
# global feature importance
global_tab_explanation = tab_explainer.explain_global(X_train)
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
# local feature importance
local_tab_explanation = tab_explainer.explain_local(X_test[0:5])
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()


# PFIExplainer
from interpret.ext.blackbox import PFIExplainer

pfi_explainer = PFIExplainer(
    model=loan_model,
    features = ['loan_amount','income','age','marital_status'],
    classes=['reject','approve']
)
# global feature importance
# The code is the same for MimicExplainer and TabularExplainer. The PFIExplainer requires the actual labels that correspond to the test features.
global_pfi_explanation = pfi_explainer.explain_global(X_train, y_train)
global_pfi_feature_importance = global_pfi_explanation.get_feature_importance_dict()

# The PFIExplainer doesn't support local feature importance explanations.

## Creating explanations
When you create an estimator/script in an AML Experiment, you can create an explainer that uploads its explanation to the run for later analysis.

### Creating an explanation in the experiment script
you'll need to have the following packages:
- azureml-interpret
- azureml-contrib-interpret
make sure you have these installed in your run environment

example code:

In [ ]:
from azureml.core.run import Run 
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient 
from interpret.ext.blackbox import TabularExplainer 

run = Run.get_context()

# code to train mode goes here.... 

# get explanations - tabularexplainer based on SHAP
explainer = TabularExplainer(
    model,
    X_train,
    features=features,
    classes=labels
)
explanation = explainer.explain_global(X_test)

# get explanation client and upload the explanations
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(
    explanation,
    comment = 'Tabular Explanation'
)

run.complete()

You can view explanations in AML Studio, or use the **ExplanationClient** object:

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient 

client = ExplanationClient.from_run_id(
    workspace=ws,
    experiment_name=experiment.experiment_name,
    run_id=run.id
)
explanation = client.download_model_explanation()
feature_importances = explanation.get_feature_importance_dict()